In [ ]:
import geopandas as gpd
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

from adjustText import adjust_text
from scipy.stats import kendalltau, pearsonr

In [ ]:
country = "Chile"
year = 2021
location_level = "province"

colors = json.load(open("consts.json", encoding="utf-8"))

In [ ]:
regions = {
    "DE ARICA Y PARINACOTA": "Región de Arica y Parinacota",
    "DE TARAPACA": "Región de Tarapacá",
    "DE ANTOFAGASTA": "Región de Antofagasta",
    "DE ATACAMA": "Región de Atacama",
    "DE COQUIMBO": "Región de Coquimbo",
    "DE VALPARAISO": "Región de Valparaíso",
    "METROPOLITANA DE SANTIAGO": "Región Metropolitana de Santiago",
    "DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS": "Región del Libertador Bernardo O'Higgins",
    "DEL MAULE": "Región del Maule",
    "DE ÑUBLE": "Región de Ñuble",
    "DEL BIOBIO": "Región del Bío-Bío",
    "DE LA ARAUCANIA": "Región de La Araucanía",
    "DE LOS RIOS": "Región de Los Ríos",
    "DE LOS LAGOS": "Región de Los Lagos",
    "DE AYSEN DEL GENERAL CARLOS IBAÑEZ DEL CAMPO": "Región de Aysén del Gral.Ibañez del Campo",
    "DE MAGALLANES Y DE LA ANTARTICA CHILENA": "Región de Magallanes y Antártica Chilena"
}

In [ ]:
gdf = gpd.read_file("geo_shapes/Chile_provinces.zip")
gdf["Provincia"] = gdf["Provincia"].str.upper().str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")
gdf = gdf.rename(columns={"Provincia": "province"})
gdf.head()

In [ ]:
# gdf = gpd.read_file("https://raw.githubusercontent.com/fcortes/Chile-GeoJSON/master/comunas.geojson")
# gdf.head()
# gdf["Comuna"] = gdf["Comuna"].str.upper().str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")
# gdf = gdf.rename(columns={"Comuna": "commune"})

In [ ]:
df = pd.read_csv(f"data_output/{country}/{year}_divisiveness_{location_level}.csv.gz", compression="gzip")

if location_level == "province":
    df["province"] = df["province"].replace({
        "BIOBIO": "BIO-BIO",
        "CHAÑARAL": "CHANARAL",
        "DEL TAMARUGAL": "TAMARUGAL",
        "SAN FELIPE DE ACONCAGUA": "SAN FELIPE"
    })
df.head()

In [ ]:
# set(df_fround[location_level].unique()) - set(gdf[location_level].unique())

In [ ]:
# set(gdf[location_level].unique()) - set(df_fround[location_level].unique())

In [ ]:
# pd.read_csv(f"data_output/{country}/{year}_first_round.csv.gz", compression="gzip")

In [ ]:
df_fround = pd.read_csv(f"data_output/{country}/{year}_first_round.csv.gz", compression="gzip")
df_fround.columns = [x.lower() for x in df_fround.columns]

df_runoff = pd.read_csv(f"data_output/{country}/{year}_runoff.csv.gz", compression="gzip")
df_runoff.columns = [x.lower() for x in df_runoff.columns]

candidate_a, candidate_b = df_runoff.candidate.unique()
df_location = pd.read_csv(f"data_output/{country}/{year}_first_round_location.csv.gz", compression="gzip")

df_fround = pd.merge(df_fround, df_location, on="polling_id")
df_runoff = pd.merge(df_runoff, df_location, on="polling_id")

if location_level == "province":
    df_fround["province"] = df_fround["province"].replace({
        "BIOBIO": "BIO-BIO",
        "CHAÑARAL": "CHANARAL",
        "DEL TAMARUGAL": "TAMARUGAL",
        "SAN FELIPE DE ACONCAGUA": "SAN FELIPE"
    })
    
    df_runoff["province"] = df_runoff["province"].replace({
        "BIOBIO": "BIO-BIO",
        "CHAÑARAL": "CHANARAL",
        "DEL TAMARUGAL": "TAMARUGAL",
        "SAN FELIPE DE ACONCAGUA": "SAN FELIPE"
    })

In [ ]:
data = pd.merge(gdf, df, on=location_level)
data.head()

In [ ]:
fig, ax = plt.subplots(figsize=(4, 20))

data_tmp = data[data["candidate"] == "JOSE ANTONIO KAST RIST"]
data_tmp = data_tmp[~data_tmp["province"].isin(["ISLA DE PASCUA", "JUAN FERNANDEZ"])]
data_tmp.plot(
    column="value", cmap="plasma", ax=ax, legend=True,
             legend_kwds=dict(
     shrink=0.3
    ))

plt.axis("off")

In [ ]:
# df_fround["rank"] = df_fround.groupby([location_level, "candidate"]).agg({"value": "sum"}).groupby(level=0)["value"].rank(ascending=False).astype(int)
#df_fround.groupby(location_level)["value"].rank(ascending=False).astype(int)

fig, ax = plt.subplots(figsize=(10, 10), dpi=100)
dd = df_runoff.groupby([location_level, "candidate"]).agg({"value": "sum"})
dd["rank"] = dd.groupby(level=0)["value"].rank(ascending=False).astype(int)
dd = dd.reset_index()

data_tmp = dd[dd["rank"] == 1]
data_tmp = pd.merge(data_tmp, gdf, on=location_level)
data_tmp = data_tmp[~data_tmp["province"].isin(["ISLA DE PASCUA", "JUAN FERNANDEZ"])]
data_tmp["color"] = data_tmp["candidate"].replace(colors)

data_tmp = gpd.GeoDataFrame(data_tmp)
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# data_tmp["value"] = NormalizeData(data_tmp["value"])

data_tmp.plot(
    column="candidate",
    color=data_tmp["color"],
    edgecolor="black",
    lw=0.2,
    ax=ax, 
    legend=True, 
    legend_kwds=dict(
#         shrink=0.3
    )
)

plt.axis("off")

In [ ]:
df["rank"] = df.groupby(location_level)["value"].rank(ascending=False).astype(int)

fig, ax = plt.subplots(figsize=(10, 10), dpi=100)

data_tmp = df[df["rank"] == 1]
data_tmp = pd.merge(data_tmp, gdf, on=location_level)
data_tmp = data_tmp[~data_tmp["province"].isin(["ISLA DE PASCUA", "JUAN FERNANDEZ"])]
data_tmp["color"] = data_tmp["candidate"].replace(colors)

data_tmp = gpd.GeoDataFrame(data_tmp)
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# data_tmp["value"] = NormalizeData(data_tmp["value"])

data_tmp.plot(
    column="candidate",
    color=data_tmp["color"],
    edgecolor="black",
    lw=0.2,
    ax=ax, 
    legend=True, 
    legend_kwds=dict(
#         shrink=0.3
    )
)

plt.axis("off")

In [ ]:
#Imports
import matplotlib.patches as mpatches

# where some data has already been plotted to ax
handles, labels = ax.get_legend_handles_labels()

for key in colors.keys():
    if key in df["candidate"].unique():
        value = colors[key]
        patch = mpatches.Patch(color=value, label=key)
        handles.append(patch) 
        
legend = plt.legend(handles=handles, loc='lower center', fontsize=24, frameon=False, ncol=len(handles))

def export_legend(legend, filename="legend.png"):
    fig  = legend.figure
    fig.canvas.draw()
    bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    # fig.savefig(filename, dpi="figure", bbox_inches=bbox)

plt.axis("off")
# export_legend(legend)
# plt.show()
legend

In [ ]:


#Imports
import matplotlib.patches as mpatches

# where some data has already been plotted to ax
handles, labels = ax.get_legend_handles_labels()

# manually define a new patch 
patch = mpatches.Patch(color='grey', label='Manual Label')

# handles is a list, so append manual patch
handles.append(patch) 

# plot the legend
plt.legend(handles=handles, loc='upper center')

In [ ]:
endorsements = {
    "JOSE ANTONIO KAST RIST": [("SEBASTIAN SICHEL RAMIREZ", "FRANCO PARISI FERNANDEZ"), ()],
    "GABRIEL BORIC FONT": [("MARCO ENRIQUEZ-OMINAMI GUMUCIO", "YASNA PROVOSTE CAMPILLAY"), ()]
}

aa = df_fround[df_fround["candidate"].isin(endorsements[candidate_a][0])].groupby("polling_id").agg({"rate": "sum"})
aa = aa.reset_index()
aa = aa.rename(columns={"rate": candidate_a})

bb = df_fround[df_fround["candidate"].isin(endorsements[candidate_b][0])].groupby("polling_id").agg({"rate": "sum"})
bb = bb.reset_index()
bb = bb.rename(columns={"rate": candidate_b})


df_endorsement = pd.merge(aa, bb, on="polling_id")
df_endorsement

In [ ]:
aa = df_fround.groupby("polling_id").agg({"value": "sum"}).reset_index()
bb = df_runoff.groupby("polling_id").agg({"value": "sum"}).reset_index()
cc = df_location.groupby("polling_id").agg({"voters": "sum"}).reset_index()
aa = pd.merge(aa, cc, on="polling_id")
aa["participation"] = aa["value"] / aa["voters"]
bb = pd.merge(bb, cc, on="polling_id")
bb["participation"] = bb["value"] / bb["voters"]
cc = pd.merge(aa, bb, on="polling_id")
cc["diff_participation"] = cc["participation_y"] - cc["participation_x"]
cc["increase_participation"] = cc["diff_participation"] > 0
cc["increase_participation"] = cc["increase_participation"].astype(int)

In [ ]:
cc.increase_participation.sum()
cc.shape

In [ ]:
dd = pd.merge(df_fround, df_runoff, on=["polling_id", "candidate"])
dd = dd.pivot_table(index="polling_id", columns=["candidate"], values=["rate_x", "rate_y"])
dd = dd.reset_index()
dd.columns = [f"{a}{b}" for a, b in dd.columns]

dd["winner"] = dd.apply(lambda x: \
                        candidate_a if x[f"rate_x{candidate_a}"] > x[f"rate_x{candidate_b}"] else candidate_b, axis=1)
dd["loser"] = dd.apply(lambda x: \
                        candidate_a if x[f"rate_x{candidate_a}"] < x[f"rate_x{candidate_b}"] else candidate_b, axis=1)

dd["rate_winner"] = dd.apply(lambda x: \
                        x[f"rate_x{candidate_a}"] if x[f"rate_x{candidate_a}"] > x[f"rate_x{candidate_b}"] else x[f"rate_x{candidate_b}"], axis=1)
dd["rate_loser"] = dd.apply(lambda x: \
                        x[f"rate_x{candidate_a}"] if x[f"rate_x{candidate_a}"] < x[f"rate_x{candidate_b}"] else x[f"rate_x{candidate_b}"], axis=1)

dd = pd.merge(dd, df_location, on="polling_id")

dd = pd.merge(dd, df[["value", "candidate", location_level]].rename(columns={"value": "dv_winner"}), 
              left_on=[location_level, "winner"], right_on=[location_level, "candidate"])
dd = pd.merge(dd, df.rename(columns={"value": "dv_loser"}), 
    left_on=[location_level, "loser"], right_on=[location_level, "candidate"])
dd = pd.merge(dd, df_endorsement, on="polling_id")

dd["en_winner"] = dd.apply(lambda x: \
                        x[candidate_a] if x[f"rate_x{candidate_a}"] > x[f"rate_x{candidate_b}"] else x[candidate_b], axis=1)
dd["en_loser"] = dd.apply(lambda x: \
                        x[candidate_a] if x[f"rate_x{candidate_a}"] < x[f"rate_x{candidate_b}"] else x[candidate_b], axis=1)

dd["dv_diff"] = dd["dv_winner"] - dd["dv_loser"]
dd["rate_diff"] = dd["rate_winner"] - dd["rate_loser"]

dd["flip"] = ((dd[f"rate_x{candidate_a}"] > dd[f"rate_x{candidate_b}"]) & (dd[f"rate_y{candidate_a}"] < dd[f"rate_y{candidate_b}"])) |\
    ((dd[f"rate_x{candidate_b}"] > dd[f"rate_x{candidate_a}"]) & (dd[f"rate_y{candidate_b}"] < dd[f"rate_y{candidate_a}"]))

# dd = pd.merge(dd, ee, on=[location_level])
dd["flip"] = dd["flip"].astype(int)
dd = pd.merge(
    dd, 
    dd.groupby(location_level).agg({"flip": "mean"}).reset_index().rename(columns={"flip": "flip_neighbors"}),
    on=location_level
)
# dd = pd.merge(dd, cc[["polling_id", "diff_participation", "increase_participation"]], on="polling_id")
dd.to_csv("test2.csv", index=False)

In [ ]:
from data_curation import flip_df

In [ ]:
flip_df(
    df_fround,
    df_runoff,
    df_location,
    df,
    country,
    year,
    location_level
)

In [ ]:
# df_test = dd.copy()
# df_test["qdv_winner"] = pd.qcut(df_test["dv_winner"], 6).astype(str)
# df_test["qdv_loser"] = pd.qcut(df_test["dv_loser"], 6).astype(str)
# m = df_test.groupby(["qdv_winner", "qdv_loser"]).agg({"flip": "mean"}).reset_index()
# m = m.pivot(index="qdv_winner", columns="qdv_loser", values="flip")

# sns.set(font_scale=2)
# fig, ax = plt.subplots(figsize=(9.5, 8))
# g = sns.heatmap(m, cmap="plasma")
# g.invert_yaxis()
# g.set_xlabel("Divisiveness (%) Winner")
# g.set_ylabel("Divisiveness (%) Loser")
# g.set_xticklabels([])
# g.set_yticklabels([])
# g

In [ ]:
# df_test = dd.copy().dropna()
# df_test["qdv_winner"] = pd.qcut(df_test["rate_winner"], 10).astype(str)
# df_test["qdv_loser"] = pd.qcut(df_test["rate_loser"], 10).astype(str)
# m = df_test.groupby(["qdv_winner", "qdv_loser"]).agg({"flip": "mean"}).reset_index()
# m = m.pivot(index="qdv_winner", columns="qdv_loser", values="flip")

# sns.set(font_scale=2)
# fig, ax = plt.subplots(figsize=(9.5, 8))
# g = sns.heatmap(m, cmap="plasma")
# g.invert_yaxis()
# g.set_xlabel("Rate (%) Winner")
# g.set_ylabel("Rate (%) Loser")
# g.set_xticklabels([])
# g.set_yticklabels([])
# g

In [ ]:
df_geo = pd.read_excel(
    "https://repositoriodeis.minsal.cl/ContenidoSitioWeb2020/uploads/2019/11/DPA2018.xls",
    header=1
)
values = {
    "Código Comuna desde 2018": "commune_id",
    "Nombre Comuna": "commune",
    "Código Provincia desde 2018": "province_id",
    "Provincia desde 2018": "province",
    "Código Región desde 2018": "region_id",
    "Nombre Región desde 2018": "region"
}
df_geo = df_geo.rename(columns=values)
df_geo = df_geo[values.values()]
df_geo.head()

In [ ]:
df_poverty = pd.read_excel("data_external/Chile/poverty_2020.xlsx", header=2)
df_poverty = df_poverty[~df_poverty["Nombre comuna"].isna()]
# df_poverty = df_poverty.rename(columns={"Nombre comuna": "commune", "Porcentaje de personas en situación de pobreza por ingresos 2020": "poverty_rate"})
# df_poverty["commune"] = df_poverty["commune"].apply(lambda x: x.upper())
df_poverty = df_poverty.rename(columns={
    "Nombre comuna": "commune", 
    "Código": "commune_id",
    "Número de personas según proyecciones de población (*)": "population",
    "Número de personas en situación de pobreza por ingresos (**)": "poverty",
    "Porcentaje de personas en situación de pobreza por ingresos 2020": "poverty_rate"
})

In [ ]:
tmp = pd.merge(df_geo, df_poverty[["commune_id", "population", "poverty"]], on="commune_id")
tmp = tmp.groupby("province").agg({"population": "sum", "poverty": "sum"})
tmp["poverty_rate"] = tmp["poverty"] / tmp["population"]
tmp = tmp.reset_index()
tmp["province"] = tmp["province"].str.upper()
tmp.head()

In [ ]:
df_tmp = df.groupby("province").agg({"value": "mean"}).reset_index()
data = pd.merge(df_tmp, tmp, on="province")
# data = pd.merge(data, df_location[["province", "province_id"]].drop_duplicates(), on="department")
import numpy as np
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# data_tmp["value"] = NormalizeData(data_tmp["value"])

data["value"] = NormalizeData(data["value"])
sns.set(font_scale=2, style="white")
fig, ax = plt.subplots(figsize=(9, 9), dpi=100)

x = "poverty_rate"
y = "value"

corr, _ = pearsonr(data[x], data[y])
p_value = "{:0.3e}".format(_)

# plt.legend([], [], loc=2, title=f"R²={(corr*corr*100).round(1)}%\np-value={p_value}", frameon=False)
FONT_SIZE = 18
ax.text(
    0.035, 0.95, 
           f"Pearson correlation = {(corr).round(3)}", ha="left", va="center", fontsize=FONT_SIZE, 
           transform=ax.transAxes)
ax.text(0.035, 0.9, 
       f"p-value = {p_value}", ha="left", va="center", fontsize=FONT_SIZE, 
       transform=ax.transAxes)

sns.scatterplot(x="poverty_rate", s=80, lw=0, y="value", data=data, ax=ax, color="#138303")

ax.set_xlabel("Poverty Rate (%)")
ax.set_ylabel("Divisiveness (Normalized)")

ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))

# texts = []
# for x, tmp in data.iterrows():
#     texts.append(ax.text(tmp["poverty_rate"], tmp["value"], tmp["department_id"], fontsize=16, color="#444444"))
    
# adjust_text(texts, lim=200, ax=ax, arrowprops=dict(arrowstyle="-", color="#766596")) 

In [ ]:
gdf

In [ ]:
df_dv_a = pd.read_csv(f"data_output/{country}/2017_divisiveness_{location_level}.csv.gz", compression="gzip") #_{rnd}
# df_dv_a = pd.merge(df_dv_a, df_location[["department_id", "department"]], on=location_level)


df_dv_b = pd.read_csv(f"data_output/{country}/2021_divisiveness_{location_level}.csv.gz", compression="gzip") #_{rnd}
# df_dv_b = pd.merge(df_dv_b, df_location[["department_id", "department"]], on=location_level)


In [ ]:
cc = pd.merge(
    df_dv_a.groupby(location_level).agg({"value": "mean"}).reset_index(), 
    df_dv_b.groupby(location_level).agg({"value": "mean"}).reset_index(),
    on=location_level
)
gdf2 = pd.merge(gdf, cc, on=location_level)

In [ ]:
gdf2[["value_x", "value_y", "averaged_values_x", "averaged_values_y"]].head()

In [ ]:
import libpysal

# create weights
W = libpysal.weights.Queen.from_dataframe(gdf2)

# row-normalise weights
W.transform = "r"

# create lag
gdf2["averaged_values_x"] = libpysal.weights.lag_spatial(W, gdf2["value_x"])
gdf2["averaged_values_y"] = libpysal.weights.lag_spatial(W, gdf2["value_y"])

In [ ]:
gdf2[["value_x", "value_y", "averaged_values_x", "averaged_values_y"]]\
.to_csv(f"data_regressions/{country}_time_lag.csv", index=False)